## Aggregation

In [1]:
import torch
from chemprop.nn.agg import MeanAggregation, SumAggregation, NormAggregation, AttentiveAggregation

This is example output from [message passing](./message_passing.ipynb) for input to aggregation.

In [2]:
n_atoms_in_batch = 7
hidden_dim = 3
example_message_passing_output = torch.randn(n_atoms_in_batch, hidden_dim)
which_atoms_in_which_molecule = torch.tensor([0, 0, 1, 1, 1, 1, 2]).long()

### Combine nodes

The aggregation layer combines the node level represenations into a graph level representaiton (usually atoms -> molecule).

### Mean and sum aggregation 

Mean aggregation is recommended when the property to predict does not depend on the number of atoms in the molecules (intensive). Sum aggregation is recommended when the property is extensive, though usually norm aggregation is better.

In [3]:
mean_agg = MeanAggregation()
sum_agg = SumAggregation()

In [4]:
mean_agg(H=example_message_passing_output, batch=which_atoms_in_which_molecule)

tensor([[-2.8124e-01,  6.0467e-01,  6.6866e-01],
        [-6.5573e-01,  2.6423e-01, -9.1855e-04],
        [ 1.2218e+00,  9.7911e-01, -1.1803e+00]])

In [5]:
sum_agg(H=example_message_passing_output, batch=which_atoms_in_which_molecule)

tensor([[-0.5625,  1.2093,  1.3373],
        [-2.6229,  1.0569, -0.0037],
        [ 1.2218,  0.9791, -1.1803]])

### Norm aggregation

Norm aggregation can be better than sum aggregation when the molecules are large as it is best to keep the hidden representation values on the order of 1 (though this is less important when batch normalization is used). The normalization constant can be customized (defaults to 100.0).

In [6]:
norm_agg = NormAggregation()
big_norm = NormAggregation(norm=1000.0)

In [7]:
norm_agg(H=example_message_passing_output, batch=which_atoms_in_which_molecule)

tensor([[-5.6249e-03,  1.2093e-02,  1.3373e-02],
        [-2.6229e-02,  1.0569e-02, -3.6742e-05],
        [ 1.2218e-02,  9.7911e-03, -1.1803e-02]])

In [8]:
big_norm(H=example_message_passing_output, batch=which_atoms_in_which_molecule)

tensor([[-5.6249e-04,  1.2093e-03,  1.3373e-03],
        [-2.6229e-03,  1.0569e-03, -3.6742e-06],
        [ 1.2218e-03,  9.7911e-04, -1.1803e-03]])

### Attentive aggregation 

This uses a learned weighted average to combine atom representations within a molecule graph. It needs to be told the size of the hidden dimension as it uses the hidden representation of each atom to calculate the weight of that atom. 

In [9]:
att_agg = AttentiveAggregation(output_size=hidden_dim)

In [10]:
att_agg(H=example_message_passing_output, batch=which_atoms_in_which_molecule)

tensor([[-0.2131,  1.3334,  0.8866],
        [-0.9319,  0.2259,  0.1872],
        [ 1.2218,  0.9791, -1.1803]], grad_fn=<ScatterReduceBackward0>)